In [1]:
!rm -rf model_discovery
!rm -rf data
import gdown
# download source code
gdown.download('https://drive.google.com/uc?id=1exw9vOYcb0fPVseleifK0m1VxbBCsBM_', output=None, quiet=False)
# download data
gdown.download('https://drive.google.com/uc?id=1ozrvNU128WOGWDVhhDKtZENzBMuO-w4i', output=None, quiet=False)

!unzip -qq data.zip
!unzip -qq model_discovery.zip

Downloading...
From: https://drive.google.com/uc?id=1exw9vOYcb0fPVseleifK0m1VxbBCsBM_
To: /content/model_discovery.zip
100%|██████████| 11.9k/11.9k [00:00<00:00, 3.13MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ozrvNU128WOGWDVhhDKtZENzBMuO-w4i
To: /content/data.zip
22.5MB [00:00, 93.3MB/s]


In [2]:
try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher

     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 


In [3]:
import deepmatcher as dm
import pandas as pd
import os
from tqdm.notebook import tqdm
import itertools

In [4]:
# download fasttext word vector
gdown.download('https://drive.google.com/uc?id=12tdZ76mb8tT98pz3lAy2gWIPZvmKDTSo', output=None, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=12tdZ76mb8tT98pz3lAy2gWIPZvmKDTSo
To: /content/wiki.en.bin
8.49GB [01:30, 93.7MB/s]


'wiki.en.bin'

In [5]:
WIKI_EN_PATH = '/content'
DATA_PATH = 'data/magellan'

In [6]:
datasets = ['Anime', 'Bikes', 'Books1', 'Books2', 'Books3', 'Restaurants4',
      'Movies1','Movies2','Movies3','Movies4','Movies5']

In [7]:
def model_serving(src, tar):
  src_file_train = ''.join([src, '.csv'])
  src_file_valid = ''.join([src, '.csv'])
  tar_file_valid = ''.join([tar, '.csv'])

  train, validation, test = dm.data.process(
                  path=DATA_PATH,
                  train=src_file_train,
                  validation=src_file_valid,
                  test=tar_file_valid,
                  embeddings_cache_path=WIKI_EN_PATH)
  
  model = dm.MatchingModel(attr_summarizer='hybrid')
  # number of epochs can be changed to a higher value to obtain better accuracy
  train_result = model.run_train(
            train,
            validation,
            epochs=5,
            batch_size=16,
            best_save_path='hybrid_model.pth',
            pos_neg_ratio=3)
  serving_result = model.run_eval(test)

  return train_result.item(), serving_result.item()

## Serving Accuracy

In [8]:
comb = list(itertools.permutations(datasets, 2))
src_list = []
tar_list = []
src_acc_list = []
ser_acc_list = []
for src, tar in tqdm(comb):
  print(src, tar)
  train_result, serving_result = model_serving(src, tar)

  src_list.append(src)
  tar_list.append(tar)
  src_acc_list.append(train_result)
  ser_acc_list.append(serving_result)

Anime Bikes



Reading and processing data from "data/magellan/Anime.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "data/magellan/Anime.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "data/magellan/Bikes.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:770: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the document

* Number of trainable parameters: 2798703
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2611: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.8 | Load Time:    0.2 || F1:  68.14 | Prec:  52.92 | Rec:  95.65 || Ex/s: 390.71

===>  EVAL Epoch 1


...

Finished Epoch 5 || Run Time:    0.7 | Load Time:    0.1 || F1:  79.73 | Prec:  66.85 | Rec:  98.76 || Ex/s: 485.35

===>  EVAL Epoch 5


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 5 || Run Time:    0.3 | Load Time:    0.1 || F1:  82.25 | Prec:  69.86 | Rec: 100.00 || Ex/s: 864.38

* Best F1: tensor(82.2526, device='cuda:0')
Saving best model...
Done.
---------------------

Loading best model...
Training done.
===>  EVAL Epoch 5


0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 5 || Run Time:    0.2 | Load Time:    0.3 || F1:  22.96 | Prec:  13.00 | Rec:  98.11 || Ex/s: 802.13




In [ ]:
result = pd.DataFrame({'Source 1': src_list, 'Source 2': tar_list, 
              'Source Accuracy': src_acc_list, 'Serving Accuracy': ser_acc_list})

# result_df.to_csv('dm_magellan_more_scenarios.csv', index=False)

In [ ]:
result

,Source 1,Source 2,Source Accuracy,Serving Accuracy
0,Anime,Bikes,99.382706,44.827587
1,Anime,Books1,99.382706,76.567657
2,Anime,Books2,99.382706,40.274597
3,Anime,Books3,99.382706,83.289818
4,Anime,Restaurants4,99.690399,48.771267
...,...,...,...,...
105,Movies5,Restaurants4,82.960411,49.242424
106,Movies5,Movies1,82.817863,55.425220
107,Movies5,Movies2,79.016403,78.048782
108,Movies5,Movies3,81.418915,67.213120
